<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#HMM-Analysis-for-Power-Sweep-Data" data-toc-modified-id="HMM-Analysis-for-Power-Sweep-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>HMM Analysis for Power Sweep Data</a></span></li></ul></div>

# HMM Analysis for Power Sweep Data

`project_path` should be the root folder of where all the Alazar data is stored 

In [1]:
from AlazarPowerSweepData import *

%matplotlib qt

project_path = r"G:\Shared drives\Quasiparticles_2\NBR07_Oct2022\test_power_sweep" 

power_sweep_obj = AlazarPowerSweepData(project_path)

`createPdf` takes value `True` or `False`. Set to `True` if you want to visually see the downsampled IQ plots (determined by `avgTime`) to determine the **attenuation** below which the system goes non-linear. Set to `False` if the attenuation is known.

In [2]:
createPdf = False

power_sweep_obj.process_Alazar_Data(avgTime=2,plots=createPdf)

Creating figure paths.....
Reading and sorting data files.....
Data loaded without creating IQ downsampled plots....


`intTime` is the starting integration time provied to the HMM fits.
`SNRmin` is the minimum SNR cut off constraint to the HMM program.

In [ ]:
intTime=1
SNRmin=3

power_sweep_obj.start_HMM_fit(intTime=intTime, SNRmin= SNRmin)



==========	HMM ANALYSIS STARTED	==========

Attenuation below which the system goes non-linear: 4
Please Ensure the Following Attenuation Configuration is Correct:


LO_power = 20
Splitter = -3
Fixed_attenuator_1 = -20
Fixed_attenuator_2 = -30
DA_fixed = -10
Fridge_attenuation = -75
Ecosorb = -3
Press Y/y if Correct or N/n if not: y
Please Ensure the Following MetaData is Correct:


dateOfMeasurement = 20221005
Temp = 30
eccosorb = True
KandL = True
rigidCables = True
JPA = True
TWPA = True
ClearingCoupler = False
sampleRateMHz = 10
durationSeconds = 5
f0 = 4.2727
LOf = 4.27238
phi = 0.47
Press Y/y if Correct or N/n if not: y

The power to the device is -125 dBM at the chosen attenuation 4

Number of Modes you want to fit: 3
Extracted Means:
[(-32.11627821751411, -20.69843435114177), (7.7317556116018125, 12.773914065315594), (34.95102795025946, -38.72200657538804)]

Gussed Covariance:
[[[5 1]
  [1 5]]

 [[5 1]
  [1 5]]

 [[5 1]
  [1 5]]]


Starting HMM Analysis.....


HMM fitting con